In [103]:
import pandas as pd
from numpy import NaN
import numpy as np

This is just the 2022 season so far - Coaches Polls

In [174]:
dfs = pd.read_html('https://www.sports-reference.com/cbb/seasons/2022.html')

In [175]:
weekly_polls = dfs[1].droplevel([0,1], axis=1)

In [176]:
weekly_polls = weekly_polls.drop(index=[20,21])

In [177]:
weekly_polls = weekly_polls.set_index(['School','Conf'])

In [178]:
for col in weekly_polls.columns:
    weekly_polls[col] = weekly_polls[col].astype(float)

In [179]:
weekly_polls['median'] = weekly_polls.median(axis=1, skipna=True)
weekly_polls['mean'] = weekly_polls.mean(axis=1, skipna=True)

### 2022 tournament

In [ ]:
def changeDF(df, ot=False):
    cols_with_ot = ['date','time','OT','away','home','away_score','home_score','away_Q1','home_Q1','away_Q2','home_Q2','away_Q3','home_Q3','away_Q4','home_Q4','away_ot','home_ot']
    cols_no_ot = ['date','time','away','home','away_score','home_score','away_Q1','home_Q1','away_Q2','home_Q2','away_Q3','home_Q3','away_Q4','home_Q4']
    df = df.T
    df.columns = cols_with_ot if ot == True else cols_no_ot
    return df


In [ ]:
def change_to_long(df,away_col,home_col):
    return df.apply(lambda x: np.select([x['home_away']=='away',x['home_away']=='home'],[x[away_col],x[home_col]]),axis=1)

In [3]:
tournament2022 = pd.read_csv('data/2022_tournament_raw.csv')

In [4]:
ind_semi = tournament2022[tournament2022['FINAL']=='SEMI-FINALS'].index[0]
ind_quarter = tournament2022[tournament2022['FINAL']=='QUARTER-FINALS'].index[0]
ind8 = tournament2022[tournament2022['FINAL']=='1/8-FINALS'].index[0]
ind16 = tournament2022[tournament2022['FINAL']=='1/16-FINALS'].index[0]
ind32 = tournament2022[tournament2022['FINAL']=='1/32-FINALS'].index[0]
ind_playin = tournament2022[tournament2022['FINAL']=='1/64-FINALS'].index[0]
ind64 = tournament2022[tournament2022['FINAL']=='USANCAA'].index[0]


In [196]:
df1 = tournament2022.iloc[:ind_semi,:]
df2 = tournament2022.iloc[ind_semi:ind_quarter,:]
df2 = df2.rename({'FINAL':df2.head(1).FINAL.values[0]},axis=1).iloc[1:, :]
df3 = tournament2022.iloc[ind_quarter:ind8,:]
df3 = df3.rename({'FINAL':df3.head(1).FINAL.values[0]},axis=1).iloc[1:, :]
df4 = tournament2022.iloc[ind8:ind16,:]
df4 = df4.rename({'FINAL':df4.head(1).FINAL.values[0]},axis=1).iloc[1:, :]
df5 = tournament2022.iloc[ind16:ind32,:]
df5 = df5.rename({'FINAL':df5.head(1).FINAL.values[0]},axis=1).iloc[1:, :]
df6 = tournament2022.iloc[ind32:ind_playin,:]
df6 = df6.rename({'FINAL':df6.head(1).FINAL.values[0]},axis=1).iloc[1:, :]
df7 = tournament2022.iloc[ind_playin:ind64,:].rename({'FINAL':'PLAYIN'},axis=1).iloc[1:, :]
df8 = tournament2022.iloc[ind64:,:].rename({'FINAL':'1/64-FINALS'},axis=1).iloc[2:, :]
dfs = [df1,df2,df3,df4,df5,df6,df7]

In [210]:
temp = []
for df in dfs:
    offset = 0
    if df.iloc[2].values[0] == 'AOT':
        offset += 3
        temp.append(changeDF(df.iloc[:14+offset],ot=True))
    else:
        temp.append(changeDF(df.iloc[:14+offset],ot=False))
    for i in range(1, len(df) // 14):
        if df.iloc[i*14+offset+2].values[0] == 'AOT':
            offset += 3
            temp.append(changeDF(df.iloc[i*14+offset-3:(i+1)*14+offset],ot=True))

        else:
           temp.append(changeDF(df.iloc[i*14+offset:(i+1)*14+offset],ot=False))

tournament = pd.concat(temp)

In [211]:
# remove at from home team
tournament['home'] = tournament['home'].apply(lambda x:x[2:])

In [213]:
# add in win/loss
tournament['away_result'] = np.where(tournament['away_score'] > tournament['home_score'],'win','loss')
tournament['home_result'] = np.where(tournament['away_score'] > tournament['home_score'],'loss','win')

In [214]:
tournament = tournament.melt(id_vars=['date','time','away_score','home_score','away_Q1','home_Q1','away_Q2','home_Q2','away_Q3','home_Q3','away_Q4','home_Q4','OT','away_ot','home_ot','away_result','home_result'],value_vars=['away','home'],value_name='team',var_name='home_away')

In [215]:
cols_to_change = {'score':('away_score','home_score'),'score_Q1':('away_Q1','home_Q1'),'score_Q2':('away_Q2','home_Q2'),'score_Q3':('away_Q3','home_Q3'),'score_Q4':('away_Q4','home_Q4'),'score_OT':('away_ot','home_ot'),'result':('away_result','home_result')}

In [216]:
for k, v in cols_to_change.items():
    tournament[k] = change_to_long(tournament,v[0],v[1])

In [217]:
tournament = tournament.drop(['away_score','home_score','away_Q1','home_Q1','away_Q2','home_Q2','away_Q3','home_Q3','away_Q4','home_Q4','away_ot','home_ot','away_result','home_result'],axis=1)

In [208]:
#tournament.groupby('team').result.count().reset_index().merge(weekly_polls,on='team')

In [189]:
weekly_polls = weekly_polls.reset_index().rename({'School':'team'},axis=1)

In [218]:
tournament

,date,time,OT,home_away,team,score,score_Q1,score_Q2,score_Q3,score_Q4,score_OT,result
0,4-Apr,9:20 PM,NaN,away,UNC Tar Heels,69,0,40,0,29,nan,loss
1,2-Apr,8:49 PM,NaN,away,UNC Tar Heels,81,0,34,0,47,nan,win
2,2-Apr,6:09 PM,NaN,away,Villanova Wildcats,65,0,29,0,36,nan,loss
3,27-Mar,5:05 PM,NaN,away,Saint Peter's Peacocks,49,0,19,0,30,nan,loss
4,27-Mar,2:20 PM,NaN,away,Miami Hurricanes,50,0,35,0,15,nan,loss
...,...,...,...,...,...,...,...,...,...,...,...,...
129,17-Mar,12:15 PM,NaN,home,Colorado State Rams,63,0,36,0,27,nan,loss
130,16-Mar,9:10 PM,AOT,home,Rutgers Scarlet Knights,87,0,41,0,28,18,loss
131,16-Mar,6:40 PM,NaN,home,Wright State Raiders,93,0,44,0,49,nan,win
132,15-Mar,9:10 PM,NaN,home,Wyoming Cowboys,58,0,25,0,33,nan,loss


In [233]:
teams_no_wins = tournament.groupby('team')['result'].count() - tournament[tournament['result']=='loss'].groupby('team')['result'].count()

In [237]:
teams_no_wins[teams_no_wins.eq(0)]

team
Akron Zips                            0.0
Alabama Crimson Tide                  0.0
Boise State Broncos                   0.0
Bryant University                     0.0
Cal State Fullerton Titans            0.0
Chattanooga Mocs                      0.0
Colgate Raiders                       0.0
Colorado State Rams                   0.0
Davidson Wildcats                     0.0
Delaware Fightin' Blue Hens           0.0
Georgia State Panthers                0.0
Iowa Hawkeyes                         0.0
Jacksonville State Gamecocks          0.0
Kentucky Wildcats                     0.0
LSU Tigers                            0.0
Longwood Lancers                      0.0
Loyola Chicago                        0.0
Marquette Golden Eagles               0.0
Montana State Bobcats                 0.0
Norfolk State Spartans                0.0
Rutgers Scarlet Knights               0.0
San Diego State Aztecs                0.0
San Francisco Dons                    0.0
Seton Hall Pirates           

In [223]:
tournament.groupby('team').filter(lambda x: x['result'].count() == 1)

,date,time,OT,home_away,team,score,score_Q1,score_Q2,score_Q3,score_Q4,score_OT,result
0,4-Apr,9:20 PM,NaN,away,UNC Tar Heels,69,0,40,0,29,nan,loss
2,2-Apr,6:09 PM,NaN,away,Villanova Wildcats,65,0,29,0,36,nan,loss
3,27-Mar,5:05 PM,NaN,away,Saint Peter's Peacocks,49,0,19,0,30,nan,loss
4,27-Mar,2:20 PM,NaN,away,Miami Hurricanes,50,0,35,0,15,nan,loss
5,26-Mar,8:49 PM,NaN,away,Arkansas Razorbacks,69,0,33,0,36,nan,loss
...,...,...,...,...,...,...,...,...,...,...,...,...
124,17-Mar,3:10 PM,NaN,home,Iowa Hawkeyes,63,0,28,0,35,nan,loss
127,17-Mar,1:45 PM,NaN,home,Boise State Broncos,53,0,19,0,34,nan,loss
129,17-Mar,12:15 PM,NaN,home,Colorado State Rams,63,0,36,0,27,nan,loss
130,16-Mar,9:10 PM,AOT,home,Rutgers Scarlet Knights,87,0,41,0,28,18,loss


In [219]:
tournament[tournament['result']=='win'].groupby('team')['result'].count().sort_values(ascending=False).reset_index()

,team,result
0,Kansas Jayhawks,6
1,UNC Tar Heels,5
2,Villanova Wildcats,4
3,Duke Blue Devils,4
4,Houston Cougars,3
5,Arkansas Razorbacks,3
6,Miami Hurricanes,3
7,Saint Peter's Peacocks,3
8,UCLA Bruins,2
9,Texas Tech Red Raiders,2
